In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

import re  #regular expression

In [2]:
sheets_to_load = ['Germany',
                  'Italy',
                  'Poland',
                  'Switzerland',
                  'Sweden',
                  'Thailand',
                  'UK',
                  'Vietnam',
                  'Australia']

# selected_sheets = pd.read_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\Compilation-raw\SEPTEMBER\q14-compilation.xlsx', sheet_name=sheets_to_load)
# selected_sheets = pd.read_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\Compilation-raw\OCTOBER\q14-compilation.xlsx', sheet_name=sheets_to_load)
# selected_sheets = pd.read_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\Compilation-raw\NOVEMBER\q14-compilation.xlsx', sheet_name=sheets_to_load)
selected_sheets = pd.read_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\Compilation-raw\DECEMBER\q14-compilation.xlsx', sheet_name=sheets_to_load)

Germany = selected_sheets['Germany']
Italy = selected_sheets['Italy']
Poland = selected_sheets['Poland']
Switzerland = selected_sheets['Switzerland']
Sweden = selected_sheets['Sweden']
Thailand = selected_sheets['Thailand']
UK = selected_sheets['UK']
Vietnam = selected_sheets['Vietnam']
Australia = selected_sheets['Australia']

### Germany

In [3]:
Germany.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396 entries, 0 to 395
Columns: 145 entries, Response ID to Not seen/heard:ROWENTA:GM - Where have you seen, heard, or read anything about these brands recently? Select all that apply
dtypes: float64(144), int64(1)
memory usage: 448.7 KB


In [4]:
Germany.head(2) #initial dataset

,Response ID,"From a friend, family or someone else I know:BOSCH:GM - Where have you seen, heard, or read anything about these brands recently? Select all that apply","In a store:BOSCH:GM - Where have you seen, heard, or read anything about these brands recently? Select all that apply","On a brand’s own website, app or e-mail:BOSCH:GM - Where have you seen, heard, or read anything about these brands recently? Select all that apply","When searching or shopping online:BOSCH:GM - Where have you seen, heard, or read anything about these brands recently? Select all that apply","YouTube / Video:BOSCH:GM - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Social media, e.g. Facebook and/or Instagram:BOSCH:GM - Where have you seen, heard, or read anything about these brands recently? Select all that apply","On TV (live, catch up or on demand):BOSCH:GM - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Newspaper/Magazine (advertisements):BOSCH:GM - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Outdoors (e.g. poster, bus stop, LCD/screen, underground, street lamp):BOSCH:GM - Where have you seen, heard, or read anything about these brands recently? Select all that apply",...,"On a brand’s own website, app or e-mail:ROWENTA:GM - Where have you seen, heard, or read anything about these brands recently? Select all that apply","When searching or shopping online:ROWENTA:GM - Where have you seen, heard, or read anything about these brands recently? Select all that apply","YouTube / Video:ROWENTA:GM - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Social media, e.g. Facebook and/or Instagram:ROWENTA:GM - Where have you seen, heard, or read anything about these brands recently? Select all that apply","On TV (live, catch up or on demand):ROWENTA:GM - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Newspaper/Magazine (advertisements):ROWENTA:GM - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Outdoors (e.g. poster, bus stop, LCD/screen, underground, street lamp):ROWENTA:GM - Where have you seen, heard, or read anything about these brands recently? Select all that apply","At Events (fairs & exhibitions, sports, cooking, etc.):ROWENTA:GM - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Somewhere else:ROWENTA:GM - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Not seen/heard:ROWENTA:GM - Where have you seen, heard, or read anything about these brands recently? Select all that apply"
0,78230,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,78263,1.0,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Unpivot the DataFrame, renaming 'Variable' to 'Source'
unpivoted_GM = Germany.melt(id_vars=['Response ID'], var_name='Source', value_name='Brand_awareness')

# Create a duplicate of the 'cleaned_source' column named 'Source'
unpivoted_GM['cleaned_source'] = unpivoted_GM['Source']

# Rearrange columns to place 'cleaned_source' next to 'Source'
unpivoted_GM = unpivoted_GM[['Response ID', 'Source', 'cleaned_source', 'Brand_awareness']]

# Remove the unwanted suffix from the 'cleaned_source' column only
unpivoted_GM['cleaned_source'] = unpivoted_GM['cleaned_source'].str.replace(
    ":GM - Where have you seen, heard, or read anything about these brands recently? Select all that apply", 
    "", 
    regex=False
)

print(unpivoted_GM.tail())

       Response ID                                             Source  \
57019        94276  Not seen/heard:ROWENTA:GM - Where have you see...   
57020        94287  Not seen/heard:ROWENTA:GM - Where have you see...   
57021        94291  Not seen/heard:ROWENTA:GM - Where have you see...   
57022        95452  Not seen/heard:ROWENTA:GM - Where have you see...   
57023        95582  Not seen/heard:ROWENTA:GM - Where have you see...   

               cleaned_source  Brand_awareness  
57019  Not seen/heard:ROWENTA             99.0  
57020  Not seen/heard:ROWENTA              NaN  
57021  Not seen/heard:ROWENTA             99.0  
57022  Not seen/heard:ROWENTA             99.0  
57023  Not seen/heard:ROWENTA              NaN  


In [6]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'cleaned_source'
for entry in unpivoted_GM['cleaned_source']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_GM['Brand'] = brand_names
unpivoted_GM['Ad_Type'] = ad_types

In [7]:
count = unpivoted_GM['Brand_awareness'].value_counts()
print(count)

Brand_awareness
99.0    1413
2.0      831
4.0      551
7.0      524
1.0      385
6.0      349
3.0      323
8.0      312
5.0      280
9.0      230
10.0     167
97.0      77
Name: count, dtype: int64


In [8]:
# Drop the original 'cleaned_source' column
unpivoted_GM = unpivoted_GM.drop(columns=['cleaned_source'])

# Rename 'Ad_Type' to 'cleaned_source'
unpivoted_GM = unpivoted_GM.rename(columns={'Ad_Type': 'cleaned_source'})

# Rearrange columns to place 'cleaned_source' next to 'Source'
unpivoted_GM = unpivoted_GM[['Response ID', 'Source', 'cleaned_source', 'Brand_awareness', 'Brand']]

# Replace values in the 'Brand_awareness' column
unpivoted_GM['Brand_awareness'] = np.where(unpivoted_GM['Brand_awareness'].isin([1, 2, 3, 4, 5, 6, 7, 8, 9, 10]), 1, 
                        np.where(unpivoted_GM['Brand_awareness'] == 97, 97,
                        np.where(unpivoted_GM['Brand_awareness'] == 99, 99,  
                        np.where(unpivoted_GM['Brand_awareness'].isna(), 0, unpivoted_GM['Brand_awareness']))))

In [9]:
unpivoted_GM.head()

,Response ID,Source,cleaned_source,Brand_awareness,Brand
0,78230,"From a friend, family or someone else I know:B...","From a friend, family or someone else I know",0.0,BOSCH
1,78263,"From a friend, family or someone else I know:B...","From a friend, family or someone else I know",1.0,BOSCH
2,78268,"From a friend, family or someone else I know:B...","From a friend, family or someone else I know",0.0,BOSCH
3,78278,"From a friend, family or someone else I know:B...","From a friend, family or someone else I know",0.0,BOSCH
4,78299,"From a friend, family or someone else I know:B...","From a friend, family or someone else I know",1.0,BOSCH


In [10]:
count = unpivoted_GM['Brand_awareness'].value_counts()
print(count)

print("\n")

count = unpivoted_GM['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_GM)
print(f"Total number of rows: {total_rows}")

Brand_awareness
0.0     51582
1.0      3952
99.0     1413
97.0       77
Name: count, dtype: int64


Brand
BOSCH        4752
MIELE        4752
SIEMENS      4752
AEG          4752
SAMSUNG      4752
BAUKNECHT    4752
PHILIPS      4752
LG           4752
DYSON        4752
NEFF         4752
HAIER        4752
ROWENTA      4752
Name: count, dtype: int64


Total number of rows: 57024


### Sweden

In [11]:
Sweden.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Columns: 145 entries, Response ID to Not seen/heard:ROBOROCK:SW - Where have you seen, heard, or read anything about these brands recently? Select all that apply
dtypes: float64(144), int64(1)
memory usage: 453.3 KB


In [12]:
Sweden.head(2) #initial dataset

,Response ID,"From a friend, family or someone else I know:ELECTROLUX:SW - Where have you seen, heard, or read anything about these brands recently? Select all that apply","In a store:ELECTROLUX:SW - Where have you seen, heard, or read anything about these brands recently? Select all that apply","On a brand’s own website, app or e-mail:ELECTROLUX:SW - Where have you seen, heard, or read anything about these brands recently? Select all that apply","When searching or shopping online:ELECTROLUX:SW - Where have you seen, heard, or read anything about these brands recently? Select all that apply","YouTube / Video:ELECTROLUX:SW - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Social media, e.g. Facebook and/or Instagram:ELECTROLUX:SW - Where have you seen, heard, or read anything about these brands recently? Select all that apply","On TV (live, catch up or on demand):ELECTROLUX:SW - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Newspaper/Magazine (advertisements):ELECTROLUX:SW - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Outdoors (e.g. poster, bus stop, LCD/screen, underground, street lamp):ELECTROLUX:SW - Where have you seen, heard, or read anything about these brands recently? Select all that apply",...,"On a brand’s own website, app or e-mail:ROBOROCK:SW - Where have you seen, heard, or read anything about these brands recently? Select all that apply","When searching or shopping online:ROBOROCK:SW - Where have you seen, heard, or read anything about these brands recently? Select all that apply","YouTube / Video:ROBOROCK:SW - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Social media, e.g. Facebook and/or Instagram:ROBOROCK:SW - Where have you seen, heard, or read anything about these brands recently? Select all that apply","On TV (live, catch up or on demand):ROBOROCK:SW - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Newspaper/Magazine (advertisements):ROBOROCK:SW - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Outdoors (e.g. poster, bus stop, LCD/screen, underground, street lamp):ROBOROCK:SW - Where have you seen, heard, or read anything about these brands recently? Select all that apply","At Events (fairs & exhibitions, sports, cooking, etc.):ROBOROCK:SW - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Somewhere else:ROBOROCK:SW - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Not seen/heard:ROBOROCK:SW - Where have you seen, heard, or read anything about these brands recently? Select all that apply"
0,78236,NaN,2.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,78238,NaN,NaN,3.0,NaN,5.0,NaN,7.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# Unpivot the DataFrame, renaming 'Variable' to 'Source'
unpivoted_SW = Sweden.melt(id_vars=['Response ID'], var_name='Source', value_name='Brand_awareness')

# Create a duplicate of the 'cleaned_source' column named 'Source'
unpivoted_SW['cleaned_source'] = unpivoted_SW['Source']

# Rearrange columns to place 'cleaned_source' next to 'Source'
unpivoted_SW = unpivoted_SW[['Response ID', 'Source', 'cleaned_source', 'Brand_awareness']]

# Remove the unwanted suffix from the 'cleaned_source' column only
unpivoted_SW['cleaned_source'] = unpivoted_SW['cleaned_source'].str.replace(
    ":SW - Where have you seen, heard, or read anything about these brands recently? Select all that apply", 
    "", 
    regex=False
)

print(unpivoted_SW.tail())

       Response ID                                             Source  \
57595        95995  Not seen/heard:ROBOROCK:SW - Where have you se...   
57596        96005  Not seen/heard:ROBOROCK:SW - Where have you se...   
57597        96010  Not seen/heard:ROBOROCK:SW - Where have you se...   
57598        96062  Not seen/heard:ROBOROCK:SW - Where have you se...   
57599        96068  Not seen/heard:ROBOROCK:SW - Where have you se...   

                cleaned_source  Brand_awareness  
57595  Not seen/heard:ROBOROCK              NaN  
57596  Not seen/heard:ROBOROCK              NaN  
57597  Not seen/heard:ROBOROCK              NaN  
57598  Not seen/heard:ROBOROCK              NaN  
57599  Not seen/heard:ROBOROCK              NaN  


In [14]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'cleaned_source'
for entry in unpivoted_SW['cleaned_source']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_SW['Brand'] = brand_names
unpivoted_SW['Ad_Type'] = ad_types

In [15]:
count = unpivoted_SW['Brand_awareness'].value_counts()
print(count)

Brand_awareness
99.0    1305
2.0      772
7.0      540
4.0      467
6.0      410
1.0      311
5.0      267
8.0      248
3.0      243
9.0      192
10.0     155
97.0      92
Name: count, dtype: int64


In [16]:
# Drop the original 'cleaned_source' column
unpivoted_SW = unpivoted_SW.drop(columns=['cleaned_source'])

# Rename 'Ad_Type' to 'cleaned_source'
unpivoted_SW = unpivoted_SW.rename(columns={'Ad_Type': 'cleaned_source'})

# Rearrange columns to place 'cleaned_source' next to 'Source'
unpivoted_SW = unpivoted_SW[['Response ID', 'Source', 'cleaned_source', 'Brand_awareness', 'Brand']]

# Replace values in the 'Brand_awareness' column
unpivoted_SW['Brand_awareness'] = np.where(unpivoted_SW['Brand_awareness'].isin([1, 2, 3, 4, 5, 6, 7, 8, 9, 10]), 1, 
                        np.where(unpivoted_SW['Brand_awareness'] == 97, 97,
                        np.where(unpivoted_SW['Brand_awareness'] == 99, 99,  
                        np.where(unpivoted_SW['Brand_awareness'].isna(), 0, unpivoted_SW['Brand_awareness']))))

In [17]:
unpivoted_SW.head()

,Response ID,Source,cleaned_source,Brand_awareness,Brand
0,78236,"From a friend, family or someone else I know:E...","From a friend, family or someone else I know",0.0,ELECTROLUX
1,78238,"From a friend, family or someone else I know:E...","From a friend, family or someone else I know",0.0,ELECTROLUX
2,78241,"From a friend, family or someone else I know:E...","From a friend, family or someone else I know",0.0,ELECTROLUX
3,78246,"From a friend, family or someone else I know:E...","From a friend, family or someone else I know",0.0,ELECTROLUX
4,78248,"From a friend, family or someone else I know:E...","From a friend, family or someone else I know",0.0,ELECTROLUX


In [18]:
count = unpivoted_SW['Brand_awareness'].value_counts()
print(count)

print("\n")

count = unpivoted_SW['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_SW)
print(f"Total number of rows: {total_rows}")

Brand_awareness
0.0     52598
1.0      3605
99.0     1305
97.0       92
Name: count, dtype: int64


Brand
ELECTROLUX    4800
BOSCH         4800
SAMSUNG       4800
MIELE         4800
PHILIPS       4800
SIEMENS       4800
CYLINDA       4800
WHIRLPOOL     4800
LG            4800
DYSON         4800
AEG           4800
ROBOROCK      4800
Name: count, dtype: int64


Total number of rows: 57600


### Italy

In [19]:
Italy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Columns: 145 entries, Response ID to Not seen/heard:SIEMENS:IT - Where have you seen, heard, or read anything about these brands recently? Select all that apply
dtypes: float64(144), int64(1)
memory usage: 451.0 KB


In [20]:
Italy.head(2) #initial dataset

,Response ID,"From a friend, family or someone else I know:SAMSUNG:IT - Where have you seen, heard, or read anything about these brands recently? Select all that apply","In a store:SAMSUNG:IT - Where have you seen, heard, or read anything about these brands recently? Select all that apply","On a brand’s own website, app or e-mail:SAMSUNG:IT - Where have you seen, heard, or read anything about these brands recently? Select all that apply","When searching or shopping online:SAMSUNG:IT - Where have you seen, heard, or read anything about these brands recently? Select all that apply","YouTube / Video:SAMSUNG:IT - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Social media, e.g. Facebook and/or Instagram:SAMSUNG:IT - Where have you seen, heard, or read anything about these brands recently? Select all that apply","On TV (live, catch up or on demand):SAMSUNG:IT - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Newspaper/Magazine (advertisements):SAMSUNG:IT - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Outdoors (e.g. poster, bus stop, LCD/screen, underground, street lamp):SAMSUNG:IT - Where have you seen, heard, or read anything about these brands recently? Select all that apply",...,"On a brand’s own website, app or e-mail:SIEMENS:IT - Where have you seen, heard, or read anything about these brands recently? Select all that apply","When searching or shopping online:SIEMENS:IT - Where have you seen, heard, or read anything about these brands recently? Select all that apply","YouTube / Video:SIEMENS:IT - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Social media, e.g. Facebook and/or Instagram:SIEMENS:IT - Where have you seen, heard, or read anything about these brands recently? Select all that apply","On TV (live, catch up or on demand):SIEMENS:IT - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Newspaper/Magazine (advertisements):SIEMENS:IT - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Outdoors (e.g. poster, bus stop, LCD/screen, underground, street lamp):SIEMENS:IT - Where have you seen, heard, or read anything about these brands recently? Select all that apply","At Events (fairs & exhibitions, sports, cooking, etc.):SIEMENS:IT - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Somewhere else:SIEMENS:IT - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Not seen/heard:SIEMENS:IT - Where have you seen, heard, or read anything about these brands recently? Select all that apply"
0,78233,1.0,2.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99.0
1,78237,NaN,2.0,NaN,NaN,NaN,NaN,7.0,NaN,NaN,...,NaN,NaN,NaN,6.0,7.0,NaN,NaN,NaN,NaN,NaN


In [21]:
# Unpivot the DataFrame, renaming 'Variable' to 'Source'
unpivoted_IT = Italy.melt(id_vars=['Response ID'], var_name='Source', value_name='Brand_awareness')

# Create a duplicate of the 'cleaned_source' column named 'Source'
unpivoted_IT['cleaned_source'] = unpivoted_IT['Source']

# Rearrange columns to place 'cleaned_source' next to 'Source'
unpivoted_IT = unpivoted_IT[['Response ID', 'Source', 'cleaned_source', 'Brand_awareness']]

# Remove the unwanted suffix from the 'cleaned_source' column only
unpivoted_IT['cleaned_source'] = unpivoted_IT['cleaned_source'].str.replace(
    ":IT - Where have you seen, heard, or read anything about these brands recently? Select all that apply", 
    "", 
    regex=False
)

print(unpivoted_IT.tail())

       Response ID                                             Source  \
57307        93279  Not seen/heard:SIEMENS:IT - Where have you see...   
57308        93284  Not seen/heard:SIEMENS:IT - Where have you see...   
57309        93290  Not seen/heard:SIEMENS:IT - Where have you see...   
57310        93341  Not seen/heard:SIEMENS:IT - Where have you see...   
57311        93357  Not seen/heard:SIEMENS:IT - Where have you see...   

               cleaned_source  Brand_awareness  
57307  Not seen/heard:SIEMENS              NaN  
57308  Not seen/heard:SIEMENS              NaN  
57309  Not seen/heard:SIEMENS              NaN  
57310  Not seen/heard:SIEMENS              NaN  
57311  Not seen/heard:SIEMENS              NaN  


In [22]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'cleaned_source'
for entry in unpivoted_IT['cleaned_source']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_IT['Brand'] = brand_names
unpivoted_IT['Ad_Type'] = ad_types

In [23]:
count = unpivoted_IT['Brand_awareness'].value_counts()
print(count)

Brand_awareness
2.0     1291
99.0     698
7.0      686
4.0      614
6.0      484
3.0      465
1.0      420
5.0      397
8.0      325
9.0      278
10.0     212
97.0      61
Name: count, dtype: int64


In [24]:
# Drop the original 'cleaned_source' column
unpivoted_IT = unpivoted_IT.drop(columns=['cleaned_source'])

# Rename 'Ad_Type' to 'cleaned_source'
unpivoted_IT = unpivoted_IT.rename(columns={'Ad_Type': 'cleaned_source'})

# Rearrange columns to place 'cleaned_source' next to 'Source'
unpivoted_IT = unpivoted_IT[['Response ID', 'Source', 'cleaned_source', 'Brand_awareness', 'Brand']]

# Replace values in the 'Brand_awareness' column
unpivoted_IT['Brand_awareness'] = np.where(unpivoted_IT['Brand_awareness'].isin([1, 2, 3, 4, 5, 6, 7, 8, 9, 10]), 1, 
                        np.where(unpivoted_IT['Brand_awareness'] == 97, 97,
                        np.where(unpivoted_IT['Brand_awareness'] == 99, 99,  
                        np.where(unpivoted_IT['Brand_awareness'].isna(), 0, unpivoted_IT['Brand_awareness']))))

In [25]:
unpivoted_IT.head()

,Response ID,Source,cleaned_source,Brand_awareness,Brand
0,78233,"From a friend, family or someone else I know:S...","From a friend, family or someone else I know",1.0,SAMSUNG
1,78237,"From a friend, family or someone else I know:S...","From a friend, family or someone else I know",0.0,SAMSUNG
2,78242,"From a friend, family or someone else I know:S...","From a friend, family or someone else I know",0.0,SAMSUNG
3,78300,"From a friend, family or someone else I know:S...","From a friend, family or someone else I know",0.0,SAMSUNG
4,78315,"From a friend, family or someone else I know:S...","From a friend, family or someone else I know",0.0,SAMSUNG


In [26]:
count = unpivoted_IT['Brand_awareness'].value_counts()
print(count)

print("\n")

count = unpivoted_IT['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_IT)
print(f"Total number of rows: {total_rows}")

Brand_awareness
0.0     51381
1.0      5172
99.0      698
97.0       61
Name: count, dtype: int64


Brand
SAMSUNG       4776
BOSCH         4776
WHIRLPOOL     4776
LG            4776
DYSON         4776
MIELE         4776
ELECTROLUX    4776
HOTPOINT      4776
BEKO          4776
AEG           4776
HAIER         4776
SIEMENS       4776
Name: count, dtype: int64


Total number of rows: 57312


### UK

In [27]:
UK.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396 entries, 0 to 395
Columns: 145 entries, Response ID to Not seen/heard:NEFF:UK - Where have you seen, heard, or read anything about these brands recently? Select all that apply
dtypes: float64(144), int64(1)
memory usage: 448.7 KB


In [28]:
UK.head(2) #initial dataset

,Response ID,"From a friend, family or someone else I know:SAMSUNG:UK - Where have you seen, heard, or read anything about these brands recently? Select all that apply","In a store:SAMSUNG:UK - Where have you seen, heard, or read anything about these brands recently? Select all that apply","On a brand’s own website, app or e-mail:SAMSUNG:UK - Where have you seen, heard, or read anything about these brands recently? Select all that apply","When searching or shopping online:SAMSUNG:UK - Where have you seen, heard, or read anything about these brands recently? Select all that apply","YouTube / Video:SAMSUNG:UK - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Social media, e.g. Facebook and/or Instagram:SAMSUNG:UK - Where have you seen, heard, or read anything about these brands recently? Select all that apply","On TV (live, catch up or on demand):SAMSUNG:UK - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Newspaper/Magazine (advertisements):SAMSUNG:UK - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Outdoors (e.g. poster, bus stop, LCD/screen, underground, street lamp):SAMSUNG:UK - Where have you seen, heard, or read anything about these brands recently? Select all that apply",...,"On a brand’s own website, app or e-mail:NEFF:UK - Where have you seen, heard, or read anything about these brands recently? Select all that apply","When searching or shopping online:NEFF:UK - Where have you seen, heard, or read anything about these brands recently? Select all that apply","YouTube / Video:NEFF:UK - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Social media, e.g. Facebook and/or Instagram:NEFF:UK - Where have you seen, heard, or read anything about these brands recently? Select all that apply","On TV (live, catch up or on demand):NEFF:UK - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Newspaper/Magazine (advertisements):NEFF:UK - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Outdoors (e.g. poster, bus stop, LCD/screen, underground, street lamp):NEFF:UK - Where have you seen, heard, or read anything about these brands recently? Select all that apply","At Events (fairs & exhibitions, sports, cooking, etc.):NEFF:UK - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Somewhere else:NEFF:UK - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Not seen/heard:NEFF:UK - Where have you seen, heard, or read anything about these brands recently? Select all that apply"
0,78251,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,78258,1.0,2.0,NaN,4.0,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
# Unpivot the DataFrame, renaming 'Variable' to 'Source'
unpivoted_UK = UK.melt(id_vars=['Response ID'], var_name='Source', value_name='Brand_awareness')

# Create a duplicate of the 'cleaned_source' column named 'Source'
unpivoted_UK['cleaned_source'] = unpivoted_UK['Source']

# Rearrange columns to place 'cleaned_source' next to 'Source'
unpivoted_UK = unpivoted_UK[['Response ID', 'Source', 'cleaned_source', 'Brand_awareness']]

# Remove the unwanted suffix from the 'cleaned_source' column only
unpivoted_UK['cleaned_source'] = unpivoted_UK['cleaned_source'].str.replace(
    ":UK - Where have you seen, heard, or read anything about these brands recently? Select all that apply", 
    "", 
    regex=False
)

print(unpivoted_UK.tail())

       Response ID                                             Source  \
57019        94064  Not seen/heard:NEFF:UK - Where have you seen, ...   
57020        95433  Not seen/heard:NEFF:UK - Where have you seen, ...   
57021        95591  Not seen/heard:NEFF:UK - Where have you seen, ...   
57022        95639  Not seen/heard:NEFF:UK - Where have you seen, ...   
57023        96098  Not seen/heard:NEFF:UK - Where have you seen, ...   

            cleaned_source  Brand_awareness  
57019  Not seen/heard:NEFF              NaN  
57020  Not seen/heard:NEFF              NaN  
57021  Not seen/heard:NEFF              NaN  
57022  Not seen/heard:NEFF              NaN  
57023  Not seen/heard:NEFF              NaN  


In [30]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'cleaned_source'
for entry in unpivoted_UK['cleaned_source']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_UK['Brand'] = brand_names
unpivoted_UK['Ad_Type'] = ad_types

In [31]:
count = unpivoted_UK['Brand_awareness'].value_counts()
print(count)

Brand_awareness
99.0    1521
2.0      643
4.0      618
7.0      558
6.0      413
1.0      405
3.0      357
5.0      282
8.0      271
9.0      245
10.0     169
97.0      82
Name: count, dtype: int64


In [32]:
# Drop the original 'cleaned_source' column
unpivoted_UK = unpivoted_UK.drop(columns=['cleaned_source'])

# Rename 'Ad_Type' to 'cleaned_source'
unpivoted_UK = unpivoted_UK.rename(columns={'Ad_Type': 'cleaned_source'})

# Rearrange columns to place 'cleaned_source' next to 'Source'
unpivoted_UK = unpivoted_UK[['Response ID', 'Source', 'cleaned_source', 'Brand_awareness', 'Brand']]

# Replace values in the 'Brand_awareness' column
unpivoted_UK['Brand_awareness'] = np.where(unpivoted_UK['Brand_awareness'].isin([1, 2, 3, 4, 5, 6, 7, 8, 9, 10]), 1, 
                        np.where(unpivoted_UK['Brand_awareness'] == 97, 97,
                        np.where(unpivoted_UK['Brand_awareness'] == 99, 99,  
                        np.where(unpivoted_UK['Brand_awareness'].isna(), 0, unpivoted_UK['Brand_awareness']))))

In [33]:
unpivoted_UK.head()

,Response ID,Source,cleaned_source,Brand_awareness,Brand
0,78251,"From a friend, family or someone else I know:S...","From a friend, family or someone else I know",0.0,SAMSUNG
1,78258,"From a friend, family or someone else I know:S...","From a friend, family or someone else I know",1.0,SAMSUNG
2,78259,"From a friend, family or someone else I know:S...","From a friend, family or someone else I know",0.0,SAMSUNG
3,78261,"From a friend, family or someone else I know:S...","From a friend, family or someone else I know",1.0,SAMSUNG
4,78262,"From a friend, family or someone else I know:S...","From a friend, family or someone else I know",0.0,SAMSUNG


In [34]:
count = unpivoted_UK['Brand_awareness'].value_counts()
print(count)

print("\n")

count = unpivoted_UK['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_UK)
print(f"Total number of rows: {total_rows}")

Brand_awareness
0.0     51460
1.0      3961
99.0     1521
97.0       82
Name: count, dtype: int64


Brand
SAMSUNG     4752
LG          4752
HOTPOINT    4752
BOSCH       4752
DYSON       4752
BEKO        4752
SHARK       4752
ZANUSSI     4752
MIELE       4752
AEG         4752
SIEMENS     4752
NEFF        4752
Name: count, dtype: int64


Total number of rows: 57024


### Poland

In [35]:
Poland.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Columns: 145 entries, Response ID to Not seen/heard:AEG:PO - Where have you seen, heard, or read anything about these brands recently? Select all that apply
dtypes: float64(144), int64(1)
memory usage: 453.3 KB


In [36]:
Poland.head(2) #initial dataset

,Response ID,"From a friend, family or someone else I know:SAMSUNG:PO - Where have you seen, heard, or read anything about these brands recently? Select all that apply","In a store:SAMSUNG:PO - Where have you seen, heard, or read anything about these brands recently? Select all that apply","On a brand’s own website, app or e-mail:SAMSUNG:PO - Where have you seen, heard, or read anything about these brands recently? Select all that apply","When searching or shopping online:SAMSUNG:PO - Where have you seen, heard, or read anything about these brands recently? Select all that apply","YouTube / Video:SAMSUNG:PO - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Social media, e.g. Facebook and/or Instagram:SAMSUNG:PO - Where have you seen, heard, or read anything about these brands recently? Select all that apply","On TV (live, catch up or on demand):SAMSUNG:PO - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Newspaper/Magazine (advertisements):SAMSUNG:PO - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Outdoors (e.g. poster, bus stop, LCD/screen, underground, street lamp):SAMSUNG:PO - Where have you seen, heard, or read anything about these brands recently? Select all that apply",...,"On a brand’s own website, app or e-mail:AEG:PO - Where have you seen, heard, or read anything about these brands recently? Select all that apply","When searching or shopping online:AEG:PO - Where have you seen, heard, or read anything about these brands recently? Select all that apply","YouTube / Video:AEG:PO - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Social media, e.g. Facebook and/or Instagram:AEG:PO - Where have you seen, heard, or read anything about these brands recently? Select all that apply","On TV (live, catch up or on demand):AEG:PO - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Newspaper/Magazine (advertisements):AEG:PO - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Outdoors (e.g. poster, bus stop, LCD/screen, underground, street lamp):AEG:PO - Where have you seen, heard, or read anything about these brands recently? Select all that apply","At Events (fairs & exhibitions, sports, cooking, etc.):AEG:PO - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Somewhere else:AEG:PO - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Not seen/heard:AEG:PO - Where have you seen, heard, or read anything about these brands recently? Select all that apply"
0,78295,NaN,NaN,NaN,4.0,NaN,NaN,7.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99.0
1,78297,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
# Unpivot the DataFrame, renaming 'Variable' to 'Source'
unpivoted_PO = Poland.melt(id_vars=['Response ID'], var_name='Source', value_name='Brand_awareness')

# Create a duplicate of the 'cleaned_source' column named 'Source'
unpivoted_PO['cleaned_source'] = unpivoted_PO['Source']

# Rearrange columns to place 'cleaned_source' next to 'Source'
unpivoted_PO = unpivoted_PO[['Response ID', 'Source', 'cleaned_source', 'Brand_awareness']]

# Remove the unwanted suffix from the 'cleaned_source' column only
unpivoted_PO['cleaned_source'] = unpivoted_PO['cleaned_source'].str.replace(
    ":PO - Where have you seen, heard, or read anything about these brands recently? Select all that apply", 
    "", 
    regex=False
)

print(unpivoted_PO.tail())

       Response ID                                             Source  \
57595        96000  Not seen/heard:AEG:PO - Where have you seen, h...   
57596        96002  Not seen/heard:AEG:PO - Where have you seen, h...   
57597        96003  Not seen/heard:AEG:PO - Where have you seen, h...   
57598        96016  Not seen/heard:AEG:PO - Where have you seen, h...   
57599        96017  Not seen/heard:AEG:PO - Where have you seen, h...   

           cleaned_source  Brand_awareness  
57595  Not seen/heard:AEG              NaN  
57596  Not seen/heard:AEG              NaN  
57597  Not seen/heard:AEG              NaN  
57598  Not seen/heard:AEG              NaN  
57599  Not seen/heard:AEG              NaN  


In [38]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'cleaned_source'
for entry in unpivoted_PO['cleaned_source']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_PO['Brand'] = brand_names
unpivoted_PO['Ad_Type'] = ad_types

In [39]:
count = unpivoted_PO['Brand_awareness'].value_counts()
print(count)

Brand_awareness
2.0     1036
7.0      754
4.0      752
99.0     714
6.0      566
5.0      505
1.0      436
3.0      423
8.0      326
9.0      284
10.0     224
97.0      75
Name: count, dtype: int64


In [40]:
# Drop the original 'cleaned_source' column
unpivoted_PO = unpivoted_PO.drop(columns=['cleaned_source'])

# Rename 'Ad_Type' to 'cleaned_source'
unpivoted_PO = unpivoted_PO.rename(columns={'Ad_Type': 'cleaned_source'})

# Rearrange columns to place 'cleaned_source' next to 'Source'
unpivoted_PO = unpivoted_PO[['Response ID', 'Source', 'cleaned_source', 'Brand_awareness', 'Brand']]

# Replace values in the 'Brand_awareness' column
unpivoted_PO['Brand_awareness'] = np.where(unpivoted_PO['Brand_awareness'].isin([1, 2, 3, 4, 5, 6, 7, 8, 9, 10]), 1, 
                        np.where(unpivoted_PO['Brand_awareness'] == 97, 97,
                        np.where(unpivoted_PO['Brand_awareness'] == 99, 99,  
                        np.where(unpivoted_PO['Brand_awareness'].isna(), 0, unpivoted_PO['Brand_awareness']))))

In [41]:
unpivoted_PO.head()

,Response ID,Source,cleaned_source,Brand_awareness,Brand
0,78295,"From a friend, family or someone else I know:S...","From a friend, family or someone else I know",0.0,SAMSUNG
1,78297,"From a friend, family or someone else I know:S...","From a friend, family or someone else I know",0.0,SAMSUNG
2,78309,"From a friend, family or someone else I know:S...","From a friend, family or someone else I know",0.0,SAMSUNG
3,78317,"From a friend, family or someone else I know:S...","From a friend, family or someone else I know",0.0,SAMSUNG
4,78344,"From a friend, family or someone else I know:S...","From a friend, family or someone else I know",1.0,SAMSUNG


In [42]:
count = unpivoted_PO['Brand_awareness'].value_counts()
print(count)

print("\n")

count = unpivoted_PO['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_PO)
print(f"Total number of rows: {total_rows}")

Brand_awareness
0.0     51505
1.0      5306
99.0      714
97.0       75
Name: count, dtype: int64


Brand
SAMSUNG       4800
BOSCH         4800
ELECTROLUX    4800
BEKO          4800
AMICA         4800
PHILIPS       4800
WHIRLPOOL     4800
LG            4800
DYSON         4800
HAIER         4800
MIELE         4800
AEG           4800
Name: count, dtype: int64


Total number of rows: 57600


### Thailand

In [43]:
Thailand.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 399 entries, 0 to 398
Columns: 145 entries, Response ID to Not seen/heard:DYSON:TH - Where have you seen, heard, or read anything about these brands recently? Select all that apply
dtypes: float64(144), int64(1)
memory usage: 452.1 KB


In [44]:
Thailand.head(2) #initial dataset

,Response ID,"From a friend, family or someone else I know:SAMSUNG:TH - Where have you seen, heard, or read anything about these brands recently? Select all that apply","In a store:SAMSUNG:TH - Where have you seen, heard, or read anything about these brands recently? Select all that apply","On a brand’s own website, app or e-mail:SAMSUNG:TH - Where have you seen, heard, or read anything about these brands recently? Select all that apply","When searching or shopping online:SAMSUNG:TH - Where have you seen, heard, or read anything about these brands recently? Select all that apply","YouTube / Video:SAMSUNG:TH - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Social media, e.g. Facebook and/or Instagram:SAMSUNG:TH - Where have you seen, heard, or read anything about these brands recently? Select all that apply","On TV (live, catch up or on demand):SAMSUNG:TH - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Newspaper/Magazine (advertisements):SAMSUNG:TH - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Outdoors (e.g. poster, bus stop, LCD/screen, underground, street lamp):SAMSUNG:TH - Where have you seen, heard, or read anything about these brands recently? Select all that apply",...,"On a brand’s own website, app or e-mail:DYSON:TH - Where have you seen, heard, or read anything about these brands recently? Select all that apply","When searching or shopping online:DYSON:TH - Where have you seen, heard, or read anything about these brands recently? Select all that apply","YouTube / Video:DYSON:TH - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Social media, e.g. Facebook and/or Instagram:DYSON:TH - Where have you seen, heard, or read anything about these brands recently? Select all that apply","On TV (live, catch up or on demand):DYSON:TH - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Newspaper/Magazine (advertisements):DYSON:TH - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Outdoors (e.g. poster, bus stop, LCD/screen, underground, street lamp):DYSON:TH - Where have you seen, heard, or read anything about these brands recently? Select all that apply","At Events (fairs & exhibitions, sports, cooking, etc.):DYSON:TH - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Somewhere else:DYSON:TH - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Not seen/heard:DYSON:TH - Where have you seen, heard, or read anything about these brands recently? Select all that apply"
0,78252,NaN,NaN,NaN,4.0,NaN,6.0,NaN,NaN,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,78269,1.0,2.0,3.0,4.0,5.0,6.0,7.0,NaN,9.0,...,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN


In [45]:
# Unpivot the DataFrame, renaming 'Variable' to 'Source'
unpivoted_TH = Thailand.melt(id_vars=['Response ID'], var_name='Source', value_name='Brand_awareness')

# Create a duplicate of the 'cleaned_source' column named 'Source'
unpivoted_TH['cleaned_source'] = unpivoted_TH['Source']

# Rearrange columns to place 'cleaned_source' next to 'Source'
unpivoted_TH = unpivoted_TH[['Response ID', 'Source', 'cleaned_source', 'Brand_awareness']]

# Remove the unwanted suffix from the 'cleaned_source' column only
unpivoted_TH['cleaned_source'] = unpivoted_TH['cleaned_source'].str.replace(
    ":TH - Where have you seen, heard, or read anything about these brands recently? Select all that apply", 
    "", 
    regex=False
)

print(unpivoted_TH.tail())

       Response ID                                             Source  \
57451        95265  Not seen/heard:DYSON:TH - Where have you seen,...   
57452        95350  Not seen/heard:DYSON:TH - Where have you seen,...   
57453        95488  Not seen/heard:DYSON:TH - Where have you seen,...   
57454        95492  Not seen/heard:DYSON:TH - Where have you seen,...   
57455        95815  Not seen/heard:DYSON:TH - Where have you seen,...   

             cleaned_source  Brand_awareness  
57451  Not seen/heard:DYSON             99.0  
57452  Not seen/heard:DYSON              NaN  
57453  Not seen/heard:DYSON              NaN  
57454  Not seen/heard:DYSON              NaN  
57455  Not seen/heard:DYSON              NaN  


In [46]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'cleaned_source'
for entry in unpivoted_TH['cleaned_source']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_TH['Brand'] = brand_names
unpivoted_TH['Ad_Type'] = ad_types

In [47]:
count = unpivoted_TH['Brand_awareness'].value_counts()
print(count)

Brand_awareness
6.0     1683
2.0     1675
5.0     1460
7.0     1446
4.0     1330
3.0     1112
9.0     1062
1.0      926
10.0     861
8.0      656
97.0     197
99.0      65
Name: count, dtype: int64


In [48]:
# Drop the original 'cleaned_source' column
unpivoted_TH = unpivoted_TH.drop(columns=['cleaned_source'])

# Rename 'Ad_Type' to 'cleaned_source'
unpivoted_TH = unpivoted_TH.rename(columns={'Ad_Type': 'cleaned_source'})

# Rearrange columns to place 'cleaned_source' next to 'Source'
unpivoted_TH = unpivoted_TH[['Response ID', 'Source', 'cleaned_source', 'Brand_awareness', 'Brand']]

# Replace values in the 'Brand_awareness' column
unpivoted_TH['Brand_awareness'] = np.where(unpivoted_TH['Brand_awareness'].isin([1, 2, 3, 4, 5, 6, 7, 8, 9, 10]), 1, 
                        np.where(unpivoted_TH['Brand_awareness'] == 97, 97,
                        np.where(unpivoted_TH['Brand_awareness'] == 99, 99,  
                        np.where(unpivoted_TH['Brand_awareness'].isna(), 0, unpivoted_TH['Brand_awareness']))))

In [49]:
unpivoted_TH.head()

,Response ID,Source,cleaned_source,Brand_awareness,Brand
0,78252,"From a friend, family or someone else I know:S...","From a friend, family or someone else I know",0.0,SAMSUNG
1,78269,"From a friend, family or someone else I know:S...","From a friend, family or someone else I know",1.0,SAMSUNG
2,78313,"From a friend, family or someone else I know:S...","From a friend, family or someone else I know",0.0,SAMSUNG
3,78345,"From a friend, family or someone else I know:S...","From a friend, family or someone else I know",1.0,SAMSUNG
4,78363,"From a friend, family or someone else I know:S...","From a friend, family or someone else I know",1.0,SAMSUNG


In [50]:
count = unpivoted_TH['Brand_awareness'].value_counts()
print(count)

print("\n")

count = unpivoted_TH['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_TH)
print(f"Total number of rows: {total_rows}")

Brand_awareness
0.0     44983
1.0     12211
97.0      197
99.0       65
Name: count, dtype: int64


Brand
SAMSUNG       4788
LG            4788
MITSUBISHI    4788
ELECTROLUX    4788
PANASONIC     4788
SHARP         4788
PHILIPS       4788
HITACHI       4788
HAIER         4788
TEFAL         4788
BOSCH         4788
DYSON         4788
Name: count, dtype: int64


Total number of rows: 57456


### Vietnam

In [51]:
Vietnam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Columns: 145 entries, Response ID to Not seen/heard:MIDEA:VN - Where have you seen, heard, or read anything about these brands recently? Select all that apply
dtypes: float64(144), int64(1)
memory usage: 453.3 KB


In [52]:
Vietnam.head(2) #initial dataset

,Response ID,"From a friend, family or someone else I know:SAMSUNG:VN - Where have you seen, heard, or read anything about these brands recently? Select all that apply","In a store:SAMSUNG:VN - Where have you seen, heard, or read anything about these brands recently? Select all that apply","On a brand’s own website, app or e-mail:SAMSUNG:VN - Where have you seen, heard, or read anything about these brands recently? Select all that apply","When searching or shopping online:SAMSUNG:VN - Where have you seen, heard, or read anything about these brands recently? Select all that apply","YouTube / Video:SAMSUNG:VN - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Social media, e.g. Facebook and/or Instagram:SAMSUNG:VN - Where have you seen, heard, or read anything about these brands recently? Select all that apply","On TV (live, catch up or on demand):SAMSUNG:VN - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Newspaper/Magazine (advertisements):SAMSUNG:VN - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Outdoors (e.g. poster, bus stop, LCD/screen, underground, street lamp):SAMSUNG:VN - Where have you seen, heard, or read anything about these brands recently? Select all that apply",...,"On a brand’s own website, app or e-mail:MIDEA:VN - Where have you seen, heard, or read anything about these brands recently? Select all that apply","When searching or shopping online:MIDEA:VN - Where have you seen, heard, or read anything about these brands recently? Select all that apply","YouTube / Video:MIDEA:VN - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Social media, e.g. Facebook and/or Instagram:MIDEA:VN - Where have you seen, heard, or read anything about these brands recently? Select all that apply","On TV (live, catch up or on demand):MIDEA:VN - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Newspaper/Magazine (advertisements):MIDEA:VN - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Outdoors (e.g. poster, bus stop, LCD/screen, underground, street lamp):MIDEA:VN - Where have you seen, heard, or read anything about these brands recently? Select all that apply","At Events (fairs & exhibitions, sports, cooking, etc.):MIDEA:VN - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Somewhere else:MIDEA:VN - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Not seen/heard:MIDEA:VN - Where have you seen, heard, or read anything about these brands recently? Select all that apply"
0,78271,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,78275,NaN,2.0,NaN,NaN,NaN,6.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
# Unpivot the DataFrame, renaming 'Variable' to 'Source'
unpivoted_VN = Vietnam.melt(id_vars=['Response ID'], var_name='Source', value_name='Brand_awareness')

# Create a duplicate of the 'cleaned_source' column named 'Source'
unpivoted_VN['cleaned_source'] = unpivoted_VN['Source']

# Rearrange columns to place 'cleaned_source' next to 'Source'
unpivoted_VN = unpivoted_VN[['Response ID', 'Source', 'cleaned_source', 'Brand_awareness']]

# Remove the unwanted suffix from the 'cleaned_source' column only
unpivoted_VN['cleaned_source'] = unpivoted_VN['cleaned_source'].str.replace(
    ":VN - Where have you seen, heard, or read anything about these brands recently? Select all that apply", 
    "", 
    regex=False
)

print(unpivoted_VN.tail())

       Response ID                                             Source  \
57595        95842  Not seen/heard:MIDEA:VN - Where have you seen,...   
57596        95844  Not seen/heard:MIDEA:VN - Where have you seen,...   
57597        95845  Not seen/heard:MIDEA:VN - Where have you seen,...   
57598        95848  Not seen/heard:MIDEA:VN - Where have you seen,...   
57599        95852  Not seen/heard:MIDEA:VN - Where have you seen,...   

             cleaned_source  Brand_awareness  
57595  Not seen/heard:MIDEA              NaN  
57596  Not seen/heard:MIDEA              NaN  
57597  Not seen/heard:MIDEA              NaN  
57598  Not seen/heard:MIDEA              NaN  
57599  Not seen/heard:MIDEA              NaN  


In [54]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'cleaned_source'
for entry in unpivoted_VN['cleaned_source']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_VN['Brand'] = brand_names
unpivoted_VN['Ad_Type'] = ad_types

In [55]:
count = unpivoted_VN['Brand_awareness'].value_counts()
print(count)

Brand_awareness
6.0     1265
5.0     1165
2.0     1091
4.0     1060
7.0     1038
3.0      830
9.0      817
10.0     703
1.0      672
8.0      497
97.0     131
99.0     110
Name: count, dtype: int64


In [56]:
# Drop the original 'cleaned_source' column
unpivoted_VN = unpivoted_VN.drop(columns=['cleaned_source'])

# Rename 'Ad_Type' to 'cleaned_source'
unpivoted_VN = unpivoted_VN.rename(columns={'Ad_Type': 'cleaned_source'})

# Rearrange columns to place 'cleaned_source' next to 'Source'
unpivoted_VN = unpivoted_VN[['Response ID', 'Source', 'cleaned_source', 'Brand_awareness', 'Brand']]

# Replace values in the 'Brand_awareness' column
unpivoted_VN['Brand_awareness'] = np.where(unpivoted_VN['Brand_awareness'].isin([1, 2, 3, 4, 5, 6, 7, 8, 9, 10]), 1, 
                        np.where(unpivoted_VN['Brand_awareness'] == 97, 97,
                        np.where(unpivoted_VN['Brand_awareness'] == 99, 99,  
                        np.where(unpivoted_VN['Brand_awareness'].isna(), 0, unpivoted_VN['Brand_awareness']))))

In [57]:
unpivoted_VN.head()

,Response ID,Source,cleaned_source,Brand_awareness,Brand
0,78271,"From a friend, family or someone else I know:S...","From a friend, family or someone else I know",0.0,SAMSUNG
1,78275,"From a friend, family or someone else I know:S...","From a friend, family or someone else I know",0.0,SAMSUNG
2,78287,"From a friend, family or someone else I know:S...","From a friend, family or someone else I know",0.0,SAMSUNG
3,78291,"From a friend, family or someone else I know:S...","From a friend, family or someone else I know",0.0,SAMSUNG
4,78358,"From a friend, family or someone else I know:S...","From a friend, family or someone else I know",1.0,SAMSUNG


In [58]:
count = unpivoted_VN['Brand_awareness'].value_counts()
print(count)

print("\n")

count = unpivoted_VN['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_VN)
print(f"Total number of rows: {total_rows}")

Brand_awareness
0.0     48221
1.0      9138
97.0      131
99.0      110
Name: count, dtype: int64


Brand
SAMSUNG       4800
PANASONIC     4800
LG            4800
TOSHIBA       4800
ELECTROLUX    4800
SHARP         4800
PHILIPS       4800
HITACHI       4800
XIAOMI        4800
AQUA          4800
BOSCH         4800
MIDEA         4800
Name: count, dtype: int64


Total number of rows: 57600


### Australia

In [59]:
Australia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397 entries, 0 to 396
Columns: 145 entries, Response ID to Not seen/heard:AEG:AU - Where have you seen, heard, or read anything about these brands recently? Select all that apply
dtypes: float64(144), int64(1)
memory usage: 449.9 KB


In [60]:
Australia.head(2) #initial dataset

,Response ID,"From a friend, family or someone else I know:SAMSUNG:AU - Where have you seen, heard, or read anything about these brands recently? Select all that apply","In a store:SAMSUNG:AU - Where have you seen, heard, or read anything about these brands recently? Select all that apply","On a brand’s own website, app or e-mail:SAMSUNG:AU - Where have you seen, heard, or read anything about these brands recently? Select all that apply","When searching or shopping online:SAMSUNG:AU - Where have you seen, heard, or read anything about these brands recently? Select all that apply","YouTube / Video:SAMSUNG:AU - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Social media, e.g. Facebook and/or Instagram:SAMSUNG:AU - Where have you seen, heard, or read anything about these brands recently? Select all that apply","On TV (live, catch up or on demand):SAMSUNG:AU - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Newspaper/Magazine (advertisements):SAMSUNG:AU - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Outdoors (e.g. poster, bus stop, LCD/screen, underground, street lamp):SAMSUNG:AU - Where have you seen, heard, or read anything about these brands recently? Select all that apply",...,"On a brand’s own website, app or e-mail:AEG:AU - Where have you seen, heard, or read anything about these brands recently? Select all that apply","When searching or shopping online:AEG:AU - Where have you seen, heard, or read anything about these brands recently? Select all that apply","YouTube / Video:AEG:AU - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Social media, e.g. Facebook and/or Instagram:AEG:AU - Where have you seen, heard, or read anything about these brands recently? Select all that apply","On TV (live, catch up or on demand):AEG:AU - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Newspaper/Magazine (advertisements):AEG:AU - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Outdoors (e.g. poster, bus stop, LCD/screen, underground, street lamp):AEG:AU - Where have you seen, heard, or read anything about these brands recently? Select all that apply","At Events (fairs & exhibitions, sports, cooking, etc.):AEG:AU - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Somewhere else:AEG:AU - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Not seen/heard:AEG:AU - Where have you seen, heard, or read anything about these brands recently? Select all that apply"
0,78293,NaN,2.0,3.0,NaN,NaN,6.0,NaN,8.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,78361,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
# Unpivot the DataFrame, renaming 'Variable' to 'Source'
unpivoted_AU = Australia.melt(id_vars=['Response ID'], var_name='Source', value_name='Brand_awareness')

# Create a duplicate of the 'cleaned_source' column named 'Source'
unpivoted_AU['cleaned_source'] = unpivoted_AU['Source']

# Rearrange columns to place 'cleaned_source' next to 'Source'
unpivoted_AU = unpivoted_AU[['Response ID', 'Source', 'cleaned_source', 'Brand_awareness']]

# Remove the unwanted suffix from the 'cleaned_source' column only
unpivoted_AU['cleaned_source'] = unpivoted_AU['cleaned_source'].str.replace(
    ":AU - Where have you seen, heard, or read anything about these brands recently? Select all that apply", 
    "", 
    regex=False
)

print(unpivoted_AU.tail())

       Response ID                                             Source  \
57163        95482  Not seen/heard:AEG:AU - Where have you seen, h...   
57164        95486  Not seen/heard:AEG:AU - Where have you seen, h...   
57165        95487  Not seen/heard:AEG:AU - Where have you seen, h...   
57166        95489  Not seen/heard:AEG:AU - Where have you seen, h...   
57167        95490  Not seen/heard:AEG:AU - Where have you seen, h...   

           cleaned_source  Brand_awareness  
57163  Not seen/heard:AEG              NaN  
57164  Not seen/heard:AEG              NaN  
57165  Not seen/heard:AEG             99.0  
57166  Not seen/heard:AEG              NaN  
57167  Not seen/heard:AEG              NaN  


In [62]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'cleaned_source'
for entry in unpivoted_AU['cleaned_source']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_AU['Brand'] = brand_names
unpivoted_AU['Ad_Type'] = ad_types

In [63]:
count = unpivoted_AU['Brand_awareness'].value_counts()
print(count)

Brand_awareness
99.0    1191
2.0      932
4.0      619
7.0      615
6.0      439
3.0      346
1.0      334
5.0      334
8.0      225
9.0      211
10.0     183
97.0      77
Name: count, dtype: int64


In [64]:
# Drop the original 'cleaned_source' column
unpivoted_AU = unpivoted_AU.drop(columns=['cleaned_source'])

# Rename 'Ad_Type' to 'cleaned_source'
unpivoted_AU = unpivoted_AU.rename(columns={'Ad_Type': 'cleaned_source'})

# Rearrange columns to place 'cleaned_source' next to 'Source'
unpivoted_AU = unpivoted_AU[['Response ID', 'Source', 'cleaned_source', 'Brand_awareness', 'Brand']]

# Replace values in the 'Brand_awareness' column
unpivoted_AU['Brand_awareness'] = np.where(unpivoted_AU['Brand_awareness'].isin([1, 2, 3, 4, 5, 6, 7, 8, 9, 10]), 1, 
                        np.where(unpivoted_AU['Brand_awareness'] == 97, 97,
                        np.where(unpivoted_AU['Brand_awareness'] == 99, 99,  
                        np.where(unpivoted_AU['Brand_awareness'].isna(), 0, unpivoted_AU['Brand_awareness']))))

In [65]:
unpivoted_AU.head()

,Response ID,Source,cleaned_source,Brand_awareness,Brand
0,78293,"From a friend, family or someone else I know:S...","From a friend, family or someone else I know",0.0,SAMSUNG
1,78361,"From a friend, family or someone else I know:S...","From a friend, family or someone else I know",0.0,SAMSUNG
2,78394,"From a friend, family or someone else I know:S...","From a friend, family or someone else I know",0.0,SAMSUNG
3,78406,"From a friend, family or someone else I know:S...","From a friend, family or someone else I know",0.0,SAMSUNG
4,78429,"From a friend, family or someone else I know:S...","From a friend, family or someone else I know",0.0,SAMSUNG


In [66]:
count = unpivoted_AU['Brand_awareness'].value_counts()
print(count)

print("\n")

count = unpivoted_AU['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_AU)
print(f"Total number of rows: {total_rows}")

Brand_awareness
0.0     51662
1.0      4238
99.0     1191
97.0       77
Name: count, dtype: int64


Brand
SAMSUNG         4764
LG              4764
DYSON           4764
WESTINGHOUSE    4764
BOSCH           4764
HISENSE         4764
MIELE           4764
SMEG            4764
ELECTROLUX      4764
HAIER           4764
AEG             4764
Name: count, dtype: int64


Total number of rows: 57168


In [67]:
# Count blank (empty or NaN) values in Brand
blank_count = unpivoted_AU[unpivoted_AU['Brand'].isna() | (unpivoted_AU['Brand'] == '')].shape[0]
print("Count of blank values in 'Brand:", blank_count)

Count of blank values in 'Brand: 4764


In [68]:
# Replace NaN values in 'Brand' column with 'FISHER&PAYKEL'
unpivoted_AU['Brand'] = unpivoted_AU['Brand'].fillna('FISHER&PAYKEL')

# Check the count of NaN values after replacement
na_count = unpivoted_AU['Brand'].isna().sum()
print("Number of NaN values in Brand after replacement:", na_count)

Number of NaN values in Brand after replacement: 0


In [69]:
count = unpivoted_AU['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_AU)
print(f"Total number of rows: {total_rows}")

Brand
SAMSUNG          4764
LG               4764
DYSON            4764
WESTINGHOUSE     4764
FISHER&PAYKEL    4764
BOSCH            4764
HISENSE          4764
MIELE            4764
SMEG             4764
ELECTROLUX       4764
HAIER            4764
AEG              4764
Name: count, dtype: int64


Total number of rows: 57168


### Switzerland

In [70]:
Switzerland.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Columns: 145 entries, Response ID to Not seen/heard:SCHULTHESS:SZ - Where have you seen, heard, or read anything about these brands recently? Select all that apply
dtypes: float64(144), int64(1)
memory usage: 451.0 KB


In [71]:
Switzerland.head(2) #initial dataset

,Response ID,"From a friend, family or someone else I know:V ZUG:SZ - Where have you seen, heard, or read anything about these brands recently? Select all that apply","In a store:V ZUG:SZ - Where have you seen, heard, or read anything about these brands recently? Select all that apply","On a brand’s own website, app or e-mail:V ZUG:SZ - Where have you seen, heard, or read anything about these brands recently? Select all that apply","When searching or shopping online:V ZUG:SZ - Where have you seen, heard, or read anything about these brands recently? Select all that apply","YouTube / Video:V ZUG:SZ - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Social media, e.g. Facebook and/or Instagram:V ZUG:SZ - Where have you seen, heard, or read anything about these brands recently? Select all that apply","On TV (live, catch up or on demand):V ZUG:SZ - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Newspaper/Magazine (advertisements):V ZUG:SZ - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Outdoors (e.g. poster, bus stop, LCD/screen, underground, street lamp):V ZUG:SZ - Where have you seen, heard, or read anything about these brands recently? Select all that apply",...,"On a brand’s own website, app or e-mail:SCHULTHESS:SZ - Where have you seen, heard, or read anything about these brands recently? Select all that apply","When searching or shopping online:SCHULTHESS:SZ - Where have you seen, heard, or read anything about these brands recently? Select all that apply","YouTube / Video:SCHULTHESS:SZ - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Social media, e.g. Facebook and/or Instagram:SCHULTHESS:SZ - Where have you seen, heard, or read anything about these brands recently? Select all that apply","On TV (live, catch up or on demand):SCHULTHESS:SZ - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Newspaper/Magazine (advertisements):SCHULTHESS:SZ - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Outdoors (e.g. poster, bus stop, LCD/screen, underground, street lamp):SCHULTHESS:SZ - Where have you seen, heard, or read anything about these brands recently? Select all that apply","At Events (fairs & exhibitions, sports, cooking, etc.):SCHULTHESS:SZ - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Somewhere else:SCHULTHESS:SZ - Where have you seen, heard, or read anything about these brands recently? Select all that apply","Not seen/heard:SCHULTHESS:SZ - Where have you seen, heard, or read anything about these brands recently? Select all that apply"
0,78375,NaN,NaN,3.0,4.0,NaN,NaN,7.0,8.0,9.0,...,NaN,4.0,NaN,6.0,NaN,NaN,9.0,NaN,NaN,NaN
1,78387,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
# Unpivot the DataFrame, renaming 'Variable' to 'Source'
unpivoted_SZ = Switzerland.melt(id_vars=['Response ID'], var_name='Source', value_name='Brand_awareness')

# Create a duplicate of the 'cleaned_source' column named 'Source'
unpivoted_SZ['cleaned_source'] = unpivoted_SZ['Source']

# Rearrange columns to place 'cleaned_source' next to 'Source'
unpivoted_SZ = unpivoted_SZ[['Response ID', 'Source', 'cleaned_source', 'Brand_awareness']]

# Remove the unwanted suffix from the 'cleaned_source' column only
unpivoted_SZ['cleaned_source'] = unpivoted_SZ['cleaned_source'].str.replace(
    ":SZ - Where have you seen, heard, or read anything about these brands recently? Select all that apply", 
    "", 
    regex=False
)

print(unpivoted_SZ.tail())

       Response ID                                             Source  \
57307        95956  Not seen/heard:SCHULTHESS:SZ - Where have you ...   
57308        95963  Not seen/heard:SCHULTHESS:SZ - Where have you ...   
57309        95971  Not seen/heard:SCHULTHESS:SZ - Where have you ...   
57310        95973  Not seen/heard:SCHULTHESS:SZ - Where have you ...   
57311        96100  Not seen/heard:SCHULTHESS:SZ - Where have you ...   

                  cleaned_source  Brand_awareness  
57307  Not seen/heard:SCHULTHESS              NaN  
57308  Not seen/heard:SCHULTHESS              NaN  
57309  Not seen/heard:SCHULTHESS              NaN  
57310  Not seen/heard:SCHULTHESS              NaN  
57311  Not seen/heard:SCHULTHESS              NaN  


In [73]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'cleaned_source'
for entry in unpivoted_SZ['cleaned_source']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_SZ['Brand'] = brand_names
unpivoted_SZ['Ad_Type'] = ad_types

In [74]:
count = unpivoted_SZ['Brand_awareness'].value_counts()
print(count)

Brand_awareness
99.0    856
2.0     735
7.0     575
8.0     425
4.0     420
6.0     411
1.0     355
5.0     350
3.0     296
9.0     281
10.0    215
97.0    188
Name: count, dtype: int64


In [75]:
# Drop the original 'cleaned_source' column
unpivoted_SZ = unpivoted_SZ.drop(columns=['cleaned_source'])

# Rename 'Ad_Type' to 'cleaned_source'
unpivoted_SZ = unpivoted_SZ.rename(columns={'Ad_Type': 'cleaned_source'})

# Rearrange columns to place 'cleaned_source' next to 'Source'
unpivoted_SZ = unpivoted_SZ[['Response ID', 'Source', 'cleaned_source', 'Brand_awareness', 'Brand']]

# Replace values in the 'Brand_awareness' column
unpivoted_SZ['Brand_awareness'] = np.where(unpivoted_SZ['Brand_awareness'].isin([1, 2, 3, 4, 5, 6, 7, 8, 9, 10]), 1, 
                        np.where(unpivoted_SZ['Brand_awareness'] == 97, 97,
                        np.where(unpivoted_SZ['Brand_awareness'] == 99, 99,  
                        np.where(unpivoted_SZ['Brand_awareness'].isna(), 0, unpivoted_SZ['Brand_awareness']))))

In [76]:
unpivoted_SZ.head()

,Response ID,Source,cleaned_source,Brand_awareness,Brand
0,78375,"From a friend, family or someone else I know:V...","From a friend, family or someone else I know",0.0,None
1,78387,"From a friend, family or someone else I know:V...","From a friend, family or someone else I know",0.0,None
2,78581,"From a friend, family or someone else I know:V...","From a friend, family or someone else I know",0.0,None
3,78735,"From a friend, family or someone else I know:V...","From a friend, family or someone else I know",0.0,None
4,78849,"From a friend, family or someone else I know:V...","From a friend, family or someone else I know",0.0,None


In [77]:
count = unpivoted_SZ['Brand_awareness'].value_counts()
print(count)

print("\n")

count = unpivoted_SZ['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_SZ)
print(f"Total number of rows: {total_rows}")

Brand_awareness
0.0     52205
1.0      4063
99.0      856
97.0      188
Name: count, dtype: int64


Brand
ELECTROLUX    4776
MIELE         4776
DYSON         4776
BOSCH         4776
SAMSUNG       4776
SIEMENS       4776
PHILIPS       4776
BAUKNECHT     4776
AEG           4776
LIEBHERR      4776
SCHULTHESS    4776
Name: count, dtype: int64


Total number of rows: 57312


In [78]:
# Count blank (empty or NaN) values in Brand
blank_count = unpivoted_SZ[unpivoted_SZ['Brand'].isna() | (unpivoted_SZ['Brand'] == '')].shape[0]
print("Count of blank values in 'Brand:", blank_count)

Count of blank values in 'Brand: 4776


In [79]:
# Replace NaN values in 'Brand' column with 'V Zug'
unpivoted_SZ['Brand'] = unpivoted_SZ['Brand'].fillna('V ZUG')

# Check the count of NaN values after replacement
na_count = unpivoted_SZ['Brand'].isna().sum()
print("Number of NaN values in Brand after replacement:", na_count)

Number of NaN values in Brand after replacement: 0


In [80]:
count = unpivoted_SZ['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_SZ)
print(f"Total number of rows: {total_rows}")

Brand
V ZUG         4776
ELECTROLUX    4776
MIELE         4776
DYSON         4776
BOSCH         4776
SAMSUNG       4776
SIEMENS       4776
PHILIPS       4776
BAUKNECHT     4776
AEG           4776
LIEBHERR      4776
SCHULTHESS    4776
Name: count, dtype: int64


Total number of rows: 57312


### Concat and save them in another csv file

In [81]:
# Concatenate the melted DataFrames
combined_q14 = pd.concat([unpivoted_GM, unpivoted_SW, unpivoted_TH, unpivoted_IT, unpivoted_PO, 
                                unpivoted_SZ, unpivoted_VN, unpivoted_AU, unpivoted_UK], ignore_index=True)

# combined_q17.dropna(subset=['Response ID'], inplace=True)

combined_q14 = combined_q14.sort_values(by='Response ID', ascending=True)

# combined_q14.to_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\PROCESSED-DATA\SEPTEMBER\combined_q14.xlsx', float_format='%.0f', index=False)
# combined_q14.to_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\PROCESSED-DATA\OCTOBER\combined_q14.xlsx', float_format='%.0f', index=False)
# combined_q14.to_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\PROCESSED-DATA\NOVEMBER\combined_q14.xlsx', float_format='%.0f', index=False)
combined_q14.to_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\PROCESSED-DATA\DECEMBER\combined_q14.xlsx', float_format='%.0f', index=False)

print(combined_q14.head(2))

       Response ID                                             Source  \
36828        78230  At Events (fairs & exhibitions, sports, cookin...   
20196        78230  When searching or shopping online:SAMSUNG:GM -...   

                                          cleaned_source  Brand_awareness  \
36828  At Events (fairs & exhibitions, sports, cookin...              0.0   
20196                  When searching or shopping online              0.0   

         Brand  
36828       LG  
20196  SAMSUNG  


In [82]:
# Count blank (NaN) rows in the 'Response ID' column
blank_count = combined_q14['Response ID'].isna().sum()
print(f"Number of blank rows in 'Response ID': {blank_count}")

Number of blank rows in 'Response ID': 0


In [83]:
combined_q14.info()

<class 'pandas.core.frame.DataFrame'>
Index: 516096 entries, 36828 to 311667
Data columns (total 5 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Response ID      516096 non-null  int64  
 1   Source           516096 non-null  object 
 2   cleaned_source   516096 non-null  object 
 3   Brand_awareness  516096 non-null  float64
 4   Brand            516096 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 23.6+ MB


In [84]:
count = combined_q14['Brand'].value_counts()
print(count)

Brand
SAMSUNG          43008
BOSCH            43008
LG               38232
DYSON            38208
ELECTROLUX       33504
AEG              33420
MIELE            33420
PHILIPS          28716
HAIER            23880
SIEMENS          23856
WHIRLPOOL        14376
BEKO             14328
HITACHI           9588
SHARP             9588
PANASONIC         9588
BAUKNECHT         9528
HOTPOINT          9528
NEFF              9504
TOSHIBA           4800
CYLINDA           4800
ROBOROCK          4800
XIAOMI            4800
MIDEA             4800
AQUA              4800
AMICA             4800
MITSUBISHI        4788
TEFAL             4788
V ZUG             4776
SCHULTHESS        4776
LIEBHERR          4776
FISHER&PAYKEL     4764
WESTINGHOUSE      4764
HISENSE           4764
SMEG              4764
ZANUSSI           4752
ROWENTA           4752
SHARK             4752
Name: count, dtype: int64
